In [1]:
%matplotlib notebook
def show_results():

    import pandas as pd
    import matplotlib.pyplot as plt
    import ipywidgets as widgets

    from IPython.display import display
    from rdkit.Chem import PandasTools


    layout = widgets.Layout(width='100%', height='600px')

    text = widgets.Textarea(
        value='',
        placeholder='',
        description='Input:',
        disabled=False,
        layout=widgets.Layout(width='100%'),
        continuous_update=True
    )
    display(text)
    button = widgets.Button(
        value=False,
        description='Parse Text',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Parse the text from the input box',
        icon=''
    )
    display(button)
    tab_titles = ['Intermediates', 'Final']
    slider = widgets.SelectionSlider(
                options=[''],
                description='Step',
                disabled=True,
                continuous_update=False,
                orientation='horizontal',
                readout=True
                )

    intermediate_output = widgets.Output(layout=layout)
    final_output = widgets.Output(layout=layout)
    children = [widgets.VBox([slider,intermediate_output]),widgets.VBox([final_output])]
    tab = widgets.Tab()
    tab.children = children
    for i, title in enumerate(tab_titles):
        tab.set_title(i, title)
    display(tab)


    intermediates = {}

    final = {}

    def parse_intermediate_output(output):
        step = int(output[0].split()[1])
        names = ["Agent","Prior","Target","Score","SMILES"]
        data = {name:[] for name in names}
        for i in range(2,len(output)):
            values = output[i].split()
            for j in range(len(values)):
                data[names[j]].append(values[j])
        df = pd.DataFrame(data)
        PandasTools.AddMoleculeColumnToFrame(df,'SMILES','Molecule',includeFingerprints=False)
        intermediates[step] = df

        slider.options = sorted(list(intermediates.keys()))
        slider.disabled = False
        handle_slider(slider, step)

    def parse_endresult(output):
        names = ["Score","Prior log P","SMILES"]
        data = {name:[] for name in names}
        for i in range(2,len(output)):
            values = output[i].split()
            for j in range(len(values)):
                data[names[j]].append(values[j])
        df = pd.DataFrame(data)
        PandasTools.AddMoleculeColumnToFrame(df,'SMILES','Molecule',includeFingerprints=False)
        final[0] = df
        tab.children[1].children[0].clear_output()
        with tab.children[1].children[0]:
            display(df)

    def handle_text(sender):
        input_ = text.value.splitlines()
        input_ = [line.lstrip().rstrip() for line in input_] 
        n_lines = len(input_)
        for i in range(n_lines):
            if input_[i].startswith("Step"):
                end = min(i+12, n_lines-1)
                #we need at least 3 lines
                if end >= i+3:
                    parse_intermediate_output(input_[i:end])
            elif input_[i].startswith("Score     Prior log P"):
                end = min(i+52, n_lines-1)
                if end >= i+3:
                    parse_endresult(input_[i:end])
        text.value = ""

    def handle_slider(sender, last_added=None):
        value = slider.value
        if value == '':
            value = last_added
        tab.children[0].children[1].clear_output()
        with tab.children[0].children[1]:
            display(intermediates[value])


    #text.observe(handle_text, 'value')
    button.on_click(handle_text)
    slider.observe(handle_slider, names='value')
    #plt.show()
show_results()

Textarea(value='', description='Input:', layout=Layout(width='100%'), placeholder='')

Button(description='Parse Text', style=ButtonStyle(), tooltip='Parse the text from the input box')

Tab(children=(VBox(children=(SelectionSlider(continuous_update=False, description='Step', disabled=True, options=('',), value=''), Output(layout=Layout(height='600px', width='100%')))), VBox(children=(Output(layout=Layout(height='600px', width='100%')),))), _titles={'0': 'Intermediates', '1': 'Final'})

In [2]:
def show_structure():
    import ipywidgets as widgets
    from IPython.display import display
    text = widgets.Text(
        value='',
        placeholder='',
        description='Smiles:',
        disabled=False,
    )
    display(text)
    button = widgets.Button(
        value=False,
        description='Show structure',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Parse the text from the input box',
        icon=''
    )
    display(button)

    def show_3d(result):

        import py3Dmol
        from scoring_functions import ROCS_similarity

        def read_mol2(path):
            with open(path) as f:
                data = f.readlines()
            data = "".join(data)
            return data
        
        ligand = "ligands/SYK/ligand.mol2"
        pocket_mol2 = read_mol2("ligands/SYK/pocket.mol2")
        ligand_mol2 = read_mol2(ligand)
        generated_mol2 = ROCS_similarity(ligand=ligand).get_conformation(result)

        p = py3Dmol.view(width=950,height=500,linked=True,viewergrid=(1,3))

        #generated plus ligands
        p.addModel(generated_mol2, 'mol2',{"keepH":False},viewer=(0,0) )
        p.setStyle({'model':0},{'stick':  {'colorscheme':'yellowCarbon'}},viewer=(0,0))
        p.addModel(ligand_mol2, 'mol2',{"keepH":False},viewer=(0,0))
        p.setStyle({'model':1},{'stick':  {'colorscheme':'cyanCarbon'}},viewer=(0,0))
        p.zoomTo(viewer=(0,0))

        #generated in pocket
        p.addModel(generated_mol2, 'mol2',{"keepH":False},viewer=(0,1) )
        p.setStyle({'model':0},{'stick':  {'colorscheme':'yellowCarbon'}},viewer=(0,1))
        p.zoomTo(viewer=(0,1))
        p.addModel(pocket_mol2, 'mol2',{"keepH":False}, viewer=(0,1))  

        #generated plus ligands in pocket
        p.addModel(generated_mol2, 'mol2',{"keepH":False},viewer=(0,2) )
        p.setStyle({'model':0},{'stick':  {'colorscheme':'yellowCarbon'}},viewer=(0,2))
        p.addModel(ligand_mol2, 'mol2',{"keepH":False},viewer=(0,2))
        p.zoomTo(viewer=(0,2))
        p.setStyle({'model':1},{'stick':  {'colorscheme':'cyanCarbon'}},viewer=(0,2))
        p.addModel(pocket_mol2, 'mol2',{"keepH":False}, viewer=(0,2))  

        return p

    def handle_text(sender):
        input_ = text.value
        display(show_3d(input_))
    button.on_click(handle_text)
show_structure()

Text(value='', description='Smiles:', placeholder='')

Button(description='Show structure', style=ButtonStyle(), tooltip='Parse the text from the input box')